# Kernel Tuner Tutorial

## Introduction Hands-on

Welcome to the first hands-on of the Kernel Tuner tutorial. In this hands-on exercise we will learn how to **install** and access Kernel Tuner from Python, and **tune** our first CUDA kernel.

To install the latest version of `kernel_tuner` we use `pip`, but before installing it we need to have a working CUDA installation available, and be sure to have both `numpy`  and `cupy` installed.

In [ ]:
%pip install kernel_tuner

After installing all necessary packages, we can import `numpy` and `kernel_tuner`.

In [ ]:
import numpy as np
import kernel_tuner as kt

Before using Kernel Tuner, we will create a text file containing the code of the CUDA kernel that we are going to use in this hands-on.

This simple kernel is called `vector_add` and computes the elementwise sum of two vectors of size `n`.

In [ ]:
%%writefile vector_add_kernel.cu

__global__ void vector_add(float * c, float * a, float * b, int n) {
    int i = (blockIdx.x * blockDim.x) + threadIdx.x;
    
    if ( i < n ) {
        c[i] = a[i] + b[i];
    }
}

The execution of the cell above created the file `vector_add_kernel.cu` containing the source code of our kernel.

We can now use Kernel Tuner to execute the code on the GPU; please read carefully both code and comments to become familiar with how Kernel Tuner works.

For more details refer to the [API](https://benvanwerkhoven.github.io/kernel_tuner/user-api.html).

In [ ]:
# the size of the vectors
size = 1000000

# all the kernel input and output data need to use numpy data types,
# note that we explicitly state that these arrays should consist of
# 32 bit floating-point values, to match our kernel source code
a = np.random.randn(size).astype(np.float32)
b = np.random.randn(size).astype(np.float32)
c = np.zeros_like(b)
n = np.int32(size)

# now we combine these variables in an argument list, which matches
# the order and types of the function arguments of our CUDA kernel
args = [c, a, b, n]

# the next step is to create a dictionary to tell Kernel Tuner about
# the tunable parameters in our code and what values these may take
tune_params = dict()

# finally, we call tune_kernel to start the tuning process. To do so,
# we pass 
#    the name of the kernel we'd like to tune, in our case: "vector_add", 
#    the name of the file containing our source code,
#    the problem_size that our kernel operates on
#    the argument list to call our kernel function
#    the dictionary with tunable parameters
results, env = kt.tune_kernel("vector_add", "vector_add_kernel.cu", size, args, tune_params, lang="cupy")



The `tune_kernel` function returns two outputs that we saved as `results` and `env`:

* `results` is a list of dictionaries, each containing detailed information about the configurations that have been benchmarked;
* `env` is a dictionary that stores information about the hardware and software environment in which this experiment took place; it is recommended to store this information along with the benchmark results.

We can also print the content of `results` to have a look at the output.

In [ ]:
print(f"Number of configurations: {len(results)}")
print(results)

If you paid attention to the output of Kernel Tuner, you may have noticed the following message:

`UserWarning: None of the tunable parameters specify thread block dimensions!`

The reason for this message is that we have not provided any tuning parameter, we have only executed the code.

To tune `vector_add` we need to specify the values of a tunable parameter in `tune_params`.

In [ ]:
# to benchmark different configurations for the number of threads in a
# thread block, we can use the special tuning parameter "block_size_x" and
# add the values to test to it
tune_params["block_size_x"] = []

# finally, we call tune_kernel to start the tuning process
results, env = kt.tune_kernel("vector_add", "vector_add_kernel.cu", size, args, tune_params, lang="cupy")

Printing the content of `results` again we should be able to see the results of all benchmarked configurations.

In [ ]:
print(f"Number of configurations: {len(results)}")
print(results)